In [40]:
# --- Imports ---
import os
import json
import random
import pandas as pd
import time
import re
import shutil
import logging
from openai import OpenAI
import signal
from datetime import datetime


## Loading and saving data files from and into JSON or Excel files

Functions:
- load_data(path) : Loads data from an Excel or JSON file and returns it as a dictionary.
    - Args:
        path (str): the file path
    - Returns:
        dict: data loaded as a dictionary
<br>
<br>


- save_data(data, path): Saves data (dictionary or DataFrame) to a JSON or Excel file based on the file extension in the path
    - Args:
        data (dict or DataFrame): data to be saved
        path (str): file path where data will be saved
    - Returns:
        bool: True if data is successfully saved, False otherwise.
<br>
<br>

- merge_benchmarks(folder, excel_output, columns_dict):
    Creates a database from all JSON files in the specified folder and its subfolders,
    then saves it to an Excel file. If the Excel file already exists, it appends the new data to it.
    This is to create the merged excel benchmark that will be used as a base for further operations (call LLMs and classify the ouputs)

    Parameters:
    - folder (str): The path to the folder containing the JSON files.
    - excel_output (str): The file path for the Excel output.
    - columns_dict (dict): A dictionary where keys are the desired column names and values are the default values if the column doesn't exist in the JSON data.

In [11]:
def load_data(path):
    """
    Loads data from an Excel or JSON file and returns it as a dictionary.

    Args:
        path (str): the file path

    Returns:
        dict: data loaded as a dictionary
    """
    data = None

    if path.endswith('.xlsx'):
        try:
            data = pd.read_excel(path).to_dict(orient='records')
        except Exception as e:
            print(f"Error loading data from Excel file: {str(e)}")

    elif path.endswith('.json'):
        try:
            with open(path, 'r') as json_file:
                data = json.load(json_file)
        except Exception as e:
            print(f"Error loading data from JSON file: {str(e)}")
    else:
        print("Unsupported file format. Please provide an Excel (.xlsx) or JSON (.json) file.")

    return data

def save_data(data, path):
    """
    Saves data (dictionary or DataFrame) to a JSON or Excel file based on the file extension in the path

    Args:
        data (dict or DataFrame): data to be saved
        path (str): file path where data will be saved

    Returns:
        bool: True if data is successfully saved, False otherwise.
    """
    try:
        if path.endswith('.json'):
            if isinstance(data, dict):
                with open(path, 'w') as json_file:
                    json.dump(data, json_file, indent=2)
            elif isinstance(data, pd.DataFrame):
                data_dict = data.to_dict(orient='list')
                with open(path, 'w') as json_file:
                    json.dump(data_dict, json_file, indent=2)

        elif path.endswith('.xlsx'):
            if isinstance(data, dict):
                # Convert dictionary to DataFrame if it's a dictionary
                data = pd.DataFrame.from_dict(data)
            data.to_excel(path, index=False)
        else:
            print("Unsupported file format. Please provide a JSON (.json) or Excel (.xlsx) file path.")
            return False

        return True
    except Exception as e:
        print(f"Error saving data: {str(e)}")
        return False

def merge_benchmarks(folder, excel_output, columns_dict):
    """
    Creates a database from all JSON files in the specified folder and its subfolders,
    then saves it to an Excel file. If the Excel file already exists, it appends the new data to it.

    Parameters:
    - folder (str): The path to the folder containing the JSON files.
    - excel_output (str): The file path for the Excel output.
    - columns_dict (dict): A dictionary where keys are the desired column names and values are the default values if the column doesn't exist in the JSON data.
    """
    all_data = []

    # Load existing data if the Excel file exists
    if os.path.exists(excel_output):
        existing_data = load_data(excel_output)
        if existing_data:
            all_data.extend(existing_data)

    # Process new JSON files
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                data = load_data(file_path)
                if data:
                    if isinstance(data, list):
                        for item in data:
                            # Ensure specific columns are included with default values if not present
                            for col, default in columns_dict.items():
                                item.setdefault(col, default)
                            all_data.append(item)
                    elif isinstance(data, dict):
                        for item in data.values():
                            # Ensure specific columns are included with default values if not present
                            for col, default in columns_dict.items():
                                item.setdefault(col, default)
                            all_data.append(item)

    # Convert the list of dictionaries into a DataFrame
    answers_df = pd.DataFrame(all_data)

    # Save the DataFrame to an Excel file
    save_data(answers_df, excel_output)

    print(f"Excel file concatenating all answers created: {excel_output}\n ----- \n")

    return answers_df


# Query LLMs on the benchmarks

Functions:
- question_LLM_on_benchmark(LLM_function, benchmark_with_intros, answers_path):
    Gets all the LLM answers for an entire benchmark file

    Args:
        LLM_function (fct): the LLM function to be used;
        benchmark_with_intros (str): path of the benchmark
        answers_path (str): path of the answers file
<br>
<br>

- create_response_GPT_base, create_response_GPT4o, create_response_X:
    functions returning an LLM response from the selected model
<br>
<br>

- get_llm_function(LLM_name, LLM_dict=LLM_dict):
    Gets the corresponding LLM function based on the LLM name

In [7]:
def question_LLM_on_benchmark(LLM_function, benchmark_with_intros, answers_path, questions_list=None):
    """Gets all the LLM answers for a subset of questions in one benchmark file based on their positions, or all questions if questions_list is not provided.

    Args:
        LLM_function (function): the LLM function to be used; One of the following: create_response_GPT3, create_response_GPT4, create_response_Palm2
        benchmark_with_intros (str): path of the benchmark
        answers_path (str): path of the answers file
        questions_list (list, optional): list of positions of the questions to be processed. If None (default), processes all questions.
    """

    answers = {}
    data = load_data(benchmark_with_intros)

    # Determine which keys to iterate over based on questions_list
    if questions_list is None:
        keys_to_process = data.keys()
    else:
        # Ensure questions_list contains strings for keys and filter out any indices not present in the data
        keys_to_process = [str(q) for q in questions_list if str(q) in data]

    for key in keys_to_process:
        value = data[key]

        response_llm = LLM_function(value["question"])

        new_instance = {
            "question": value["question"],
            "choices": value["choices"],
            "right_answer": value["right_answer"],
            "llm_answer": response_llm
        }
        if "original_question" in value:
            new_instance["original_question"] = value["original_question"]

        answers[key] = new_instance

    save_data(answers, answers_path)


In [ ]:


OPENAI_API_KEY = "YOUR_API_KEY"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)


def create_response_GPT_base(question_prompt, basemodel, api_key=OPENAI_API_KEY):
    """Base function for getting responses from GPT models.

    Args:
        question_prompt (str): The question prompt
        basemodel (str): The model to use (e.g., "gpt-4o")
        api_key (str, optional): OpenAI API key. Defaults to OPENAI_KEY.

    Returns:
        str: The model's response
    """
    logger.info(f"--- create_response base : {basemodel} --- ")

    response = None
    retries = 0

    while retries < 3:
        try:
            client = OpenAI(api_key=api_key)

            resp = client.chat.completions.create(
                model=basemodel,
                temperature=0,
                max_tokens=1000,
                seed=1,
                messages=[
                    {"role": "user", "content": question_prompt}
                ]
            )
            print("tokens : ", resp.usage.total_tokens)
            print("model used : ", resp.model)
            response = resp.choices[0].message.content
            break
        except Exception as e:
            print(f"Error occurred: {e}")
            print(question_prompt)
            time.sleep(0.100)
            response = f"Error occurred: {e}"
            retries += 1

    return response

def create_response_GPT4o(question_prompt, api_key=OPENAI_API_KEY):
    return create_response_GPT_base(question_prompt, "gpt-4o", api_key)

def create_response_GPT3(question_prompt, api_key=OPENAI_API_KEY):
    return create_response_GPT_base(question_prompt, "gpt-3.5-turbo", api_key)

# create_response_GPT4o("what is 2+2?")

LLM_dict = {"GPT3" : {"LLM_function" :create_response_GPT3},
            "GPT4o" : {"LLM_function" :create_response_GPT4o},
                     }
                     
def get_llm_function(LLM_name, LLM_dict=LLM_dict):
    """Get the corresponding LLM function based on the LLM string.

    Args:
        llm_string (str): The LLM string.

    Returns:
        function: The corresponding LLM function.
    """
    if LLM_name in LLM_dict:
        return LLM_dict[LLM_name]["LLM_function"]
    else:
        raise ValueError(f"No LLM function found for LLM '{LLM_name}'")


## Launch on selected benchmarks

In [ ]:
Functions:
- randomly_select_lines(answers_input, answers_output, num_personas_per_category=1, selected_string="1"):
    Randomly marks  X personas per prompt_characteristic category from the answers file so that only these are processed;
    This is used specifically to launch the LLM on a subset of the answers, to save time and money, and to have a representative sample when there are multiple personas per prompt_characteristic category.
    
<br>
<br>

- process_selected_rows(excel_file_path, output_excel_file_path, model_name, randomly_selected_value="1", save_interval=10, backup_interval=100, selected_benchmarks=None, selected_prompt_characteristics=None):
    Gets LLM responses for the selected questions. This function saves the file periodically and creates backup files.



In [ ]:
def randomly_select_lines(answers_input, answers_output, num_personas_per_category=1, selected_string="1"):
    """
    Randomly marks  X personas per prompt_characteristic category from the answers file so that only these are processed;

    Args:
        answers_input (str): The path to the answers input (Excel file).
        answers_output (str): The path to the answers output (Excel file).
        num_personas_per_category (int): Number of personas to randomly select per prompt_characteristic category.
        selected_string (str): The string to add to the 'randomly_selected' column for selected lines.
    """
    # Load the excel answer file
    df = pd.read_excel(answers_input)

    # Check if the 'randomly_selected' column exists
    if 'randomly_selected' in df.columns:
        # Exclude rows that already have the 'randomly_selected' column filled with the specified string
        df = df[df['randomly_selected'].isna() | (df['randomly_selected'] != selected_string)]

    # Initialize a list to hold the indices of the selected rows
    selected_indices = []

    unique_combinations = df[['question_id', 'prompt_characteristic']].drop_duplicates()

    for _, row in unique_combinations.iterrows():
        question_id = row['question_id']
        prompt_characteristic = row['prompt_characteristic']

        # Filter the dataframe to get rows matching the current combination
        subset_df = df[(df['question_id'] == question_id) & (df['prompt_characteristic'] == prompt_characteristic)]

        # Select the specified number of random personas for this combination
        if not subset_df.empty:
            num_to_sample = min(num_personas_per_category, len(subset_df))
            selected_indices.extend(subset_df.sample(n=num_to_sample).index)

    # Update the 'randomly_selected' column for the selected lines
    df.loc[selected_indices, 'randomly_selected'] = selected_string

    # Save the updated DataFrame back to the Excel file
    df.to_excel(answers_output, index=False)
    print(f"Randomly selected personas and updated the answers file: {answers_output}")



In [ ]:

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException

processed_indices = set()

def process_selected_rows(excel_file_path, output_excel_file_path, model_name, randomly_selected_value="1", save_interval=10, backup_interval=100, selected_benchmarks=None, selected_prompt_characteristics=None):
    """
    Processes the rows in the Excel file where `randomly_selected` equals the specified value,
    and generates responses using the specified LLM model. Saves the file periodically and creates backup files.

    Args:
        excel_file_path (str): The path to the input Excel file.
        output_excel_file_path (str): The path to save the updated Excel file.
        model_name (str): The name of the LLM model to use.
        randomly_selected_value (str): The value in the `randomly_selected` column to filter by. Default is "1".
        save_interval (int): The number of rows to process before saving the file. Default is 10.
        backup_interval (int): The number of rows to process before saving a backup file. Default is 100.
        selected_benchmarks (list): List of benchmarks to filter by.
        selected_prompt_characteristics (list): List of user characteristics to filter by.
    """
    # Load the Excel file into a DataFrame
    df = pd.read_excel(excel_file_path)

    # Check if the randomly_selected column exists
    if 'randomly_selected' not in df.columns:
        raise KeyError("The 'randomly_selected' column was not found in the Excel file.")
    
    print("Contents of 'randomly_selected' column:")
    print(df['randomly_selected'].value_counts())

    # Check if the columns for the model exist, if not create them
    answer_column = f'{model_name}_answer'
    is_correct_column = f'{model_name}_is_correct'
    date_column = f'date_of_launch_{model_name}'
    
    if answer_column not in df.columns:
        df[answer_column] = None
    if is_correct_column not in df.columns:
        df[is_correct_column] = None
    if date_column not in df.columns:
        df[date_column] = None

    # If selected_benchmarks is empty, select all unique benchmarks
    if not selected_benchmarks:
        selected_benchmarks = df['benchmark'].unique().tolist()

    # If selected_prompt_characteristics is empty, select all unique user characteristics
    if not selected_prompt_characteristics:
        selected_prompt_characteristics = df['prompt_characteristic'].unique().tolist()

    # Filter rows where `randomly_selected` equals the specified value
    selected_rows = df[
        (df['randomly_selected'] == float(randomly_selected_value)) &
        (df['benchmark'].isin(selected_benchmarks)) &
        (df['prompt_characteristic'].isin(selected_prompt_characteristics))
    ]

    # Print the number of selected rows for debugging
    print(f"Number of selected rows: {len(selected_rows)}")
    unanswered_questions = selected_rows[selected_rows[answer_column].isna()]
    print(f"Number of questions left to answer: {len(unanswered_questions)}")


    # Get the corresponding LLM function
    llm_function = get_llm_function(model_name)

    # Process each selected row
    for i, (index, row) in enumerate(selected_rows.iterrows()):
        unanswered_questions = selected_rows[selected_rows[answer_column].isna()]
        unanswered_questions = selected_rows[selected_rows[answer_column].isna()]
        if index in processed_indices:
            continue
        if pd.isna(row[answer_column]):  # Only process if the answer column is empty
            question_prompt = row['question']  # Assuming there's a column 'question' with the prompt
            print(f"Processing row {i + 1}/{len(selected_rows)}")
            #print(f"Processing row {i + 1}/{len(unanswered_questions)}")

            try:
                response = llm_function(question_prompt)
            except TimeoutException:
                # Save the file before restarting
                df.to_excel(output_excel_file_path, index=False)
                return  # Exit the current function call after saving
            except Exception as e:
                print(f"Error occurred: {e}")
                response = None

            if response and isinstance(response, str) and response.startswith("Error occurred:"):
                response = None
            df.at[index, answer_column] = response
            df.at[index, is_correct_column] = "To be determined"
            df.at[index, date_column] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            # Mark the row as processed
            processed_indices.add(index)

            # Save the DataFrame every `save_interval` rows
            if (i + 1) % save_interval == 0:
                df.to_excel(output_excel_file_path, index=False)
                print(f"Saved progress to {output_excel_file_path} after processing {i + 1} rows.")
                print(f"Number of questions left to answer: {len(unanswered_questions)}")

            # Save a backup file every `backup_interval` rows
            if (i + 1) % backup_interval == 0:
                backup_file_path = output_excel_file_path.replace(".xlsx", f"_backup_{i + 1}.xlsx")
                df.to_excel(backup_file_path, index=False)
                print(f"Saved backup to {backup_file_path} after processing {i + 1} rows.")

    df.to_excel(output_excel_file_path, index=False)
    print(f"Final save completed. Updated Excel file saved to {output_excel_file_path}")


# Step 3 Main

In [ ]:

OPENAI_API_KEY = "YOUR_API_KEY"

folder_path = "YOUR_FOLDER_PATH"

# Example run
folder_modified_benchmarks_step2 = folder_path + "step2_modified-benchmarks/"
folder_answered_benchmarks_step3 = folder_path + "step3_answered-benchmarks/"
modified_benchmark_path = folder_modified_benchmarks_step2 + "strategyQA_base_base.json"
merged_benchmarks_path = folder_modified_benchmarks_step2 + "merged_benchmarks.xlsx"

LLM_dict = {"GPT3" : {"LLM_function" :create_response_GPT3},
            "GPT4o" : {"LLM_function" :create_response_GPT4o},
                     }


merge_benchmarks(folder_modified_benchmarks_step2, merged_benchmarks_path, {"benchmark_name": "", "question_id": "", "question": "", "choices": None, "right_answer": "", "original_question": "", "llm_answer": ""}) 
# here, the second part of the dictionary in argument is for the default value of the columns if there is no value indicated i the JSON files; for instance, if there are no choices, the default value for the Choices columnwill be None 

randomly_select_lines(merged_benchmarks_path, merged_benchmarks_path, num_personas_per_category=1, selected_string="1")

model_name = 'GPT4o'
process_selected_rows(merged_benchmarks_path, merged_benchmarks_path, model_name, randomly_selected_value="1")
